In [4]:
import datetime
import os
import re
import time
from tqdm import tqdm
from selenium import webdriver
import requests
import json

import os
import requests
import fuzzysearch
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import spacy

import pandas as pd
import requests_cache
from IPython.display import HTML, display
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm

s = requests_cache.CachedSession()
retries = Retry(total=10, backoff_factor=0.2, status_forcelist=[500, 502, 503, 504, 403], allowed_methods=["GET"])
s.mount("http://", HTTPAdapter(max_retries=retries))
s.mount("https://", HTTPAdapter(max_retries=retries))

/Users/fiannualamorgan/anaconda3/envs/spatial_analysis/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
api_key = "API KEY HERE"  

api_url = "https://api.trove.nla.gov.au/v3/result"

# Initialize an empty list to store all the results
all_results = []

# Initial params for the first request
params = {
    "q": "Eikon Basilike",
    "holding": "true",
    "category": "book",
    "code": "book",
    "name": "Books & Libraries",
    "encoding": "json",
    "reclevel": "full",
    "key": api_key,
    "n": 100,  # You can set this to the maximum allowed, which seems to be 100
    "include": "workversions",
    "include": "holdings",
}

while True:
    response = requests.get(api_url, params=params)
    
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.content}")
        break
    
    data = response.json()
    
    # Extract magazine_data using the corrected path
    trove_data = data.get("category", [{}])[0].get("records", {})
    
    all_results.extend(trove_data.get("work", []))
    
    # Check if there's a nextStart value to continue fetching more results
    if "nextStart" in trove_data:
        params["s"] = trove_data["nextStart"]
    else:
        break  # No more results to fetch, exit the loop

print("Total results retrieved:", len(all_results))

# Save all the results to a JSON file
with open("trove_data_dump_download.json", "w", encoding="utf-8") as json_file:
    json.dump(all_results, json_file, ensure_ascii=False, indent=4)

Total results retrieved: 247


In [2]:
import csv
import json

# Load the JSON data
with open('trove_data_dump_download.json') as f:
    data = json.load(f)

# Define the CSV file where the output will be saved
csv_file = 'WRITE FILE HERE'

# Open the CSV file for writing
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    # Define the CSV column names
    fieldnames = [
        'id', 'troveUrl', 'title', 'contributor', 'format', 'extent',
        'issued', 'holdingsCount', 'versionCount', 'nuc', 'holding_url',
        'call_number_value', 'local_identifier'
    ]

    # Create a CSV DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over each entry in the JSON data
    for entry in data:
        # Extract the relevant fields from the JSON entry
        id = entry.get('id')
        url = entry.get('troveUrl')
        title = entry.get('title')
        contributor = entry.get('contributor')  # Extracting contributor
        format_type = entry.get('format')  # Extracting format
        extent = entry.get('extent')  # Extracting extent
        issued = entry.get('issued')  # Extracting issued date
        holdings_count = entry.get('holdingsCount')
        version_count = entry.get('versionCount')

        # Iterate over each 'holding' in the entry
        for holding in entry.get('holding', []):
            nuc = holding.get('nuc')
            holding_url = holding.get('url', {}).get('value')

            # Check if there are call numbers and iterate through them
            for call_number_info in holding.get('callNumber', []):
                call_number_value = call_number_info.get('value')
                local_identifier = call_number_info.get('localIdentifier')

                # Write the row to the CSV file
                writer.writerow({
                    'id': id,
                    'troveUrl': url,
                    'title': title,
                    'contributor': contributor,
                    'format': format_type,
                    'extent': extent,
                    'issued': issued,  # Adding issued to the row
                    'holdingsCount': holdings_count,
                    'versionCount': version_count,
                    'nuc': nuc,
                    'holding_url': holding_url,
                    'call_number_value': call_number_value,
                    'local_identifier': local_identifier
                })

print(f'Data written to {csv_file}')


Data written to output_5_Nov_2.csv


In [11]:
#load in the csv file 'Edited_Eikon_Spreadsheet.csv' from current directory
df = pd.read_csv('Edited_Eikon_Spreadsheet.csv')
#count unique values in the 'id' column, print this as "there are x unique values in the id column"
print(f'There are {df["id"].nunique()} unique values in the id column for the edited spreadsheet')
#count number of unique values in the 'nuc' column
print(f'There are {df["nuc"].nunique()} unique values in the nuc column for the edited spreadsheet')


There are 154 unique values in the id column
There are 31 unique values in the nuc column


In [10]:
#load in the spreadsheet output_5_Nov_2.csv from current directory
df2 = pd.read_csv('output_5_Nov_2.csv')
#count number of unique values in the 'nuc' column
print(f'There are {df2["nuc"].nunique()} unique values in the nuc column for the original spreadsheet')
#count unique values in the 'id' column, print this as "there are x unique values in the id column"
print(f'There are {df2["id"].nunique()} unique values in the id column for the original spreadsheet')


There are 60 unique values in the nuc column
There are 206 unique values in the id column
